# Setup

## Import Library

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import os
from PIL import Image

## Bot Telegram

## Download dan Extract Datasets

In [2]:
# https://drive.google.com/file/d/1xP8hy5ZhkD_B18HCoQyFKIDs2tn-qZ6b/view?usp=sharing

# EDA

## Preview datasets

## Distribusi datasets

# Training

## Split train test

In [6]:
# Set dataset directories
train_dir = 'datasets/training'
test_dir = 'datasets/testing'

# Image parameters
img_height = 224
img_width = 224
batch_size = 32

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load MobileNetV3
base_model = MobileNetV3Small(input_shape=(img_height, img_width, 3), 
                              include_top=False, 
                              weights='imagenet')
base_model.trainable = False

# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)

# Evaluate the model
eval_result = model.evaluate(test_generator)
print(f"\nTest Accuracy: {eval_result[1] * 100:.2f}%")

# Save the model
model.save('bean_classifier_mobilenetv3.h5')

# Class indices
print("Class indices:", train_generator.class_indices)

Found 1034 images belonging to 3 classes.
Found 133 images belonging to 3 classes.
Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 21s 362ms/step - accuracy: 0.3256 - loss: 1.1297 - val_accuracy: 0.3383 - val_loss: 1.0998
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 10s 311ms/step - accuracy: 0.3224 - loss: 1.1164 - val_accuracy: 0.3383 - val_loss: 1.0978
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 10s 293ms/step - accuracy: 0.3607 - loss: 1.1017 - val_accuracy: 0.3383 - val_loss: 1.0968
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 13s 382ms/step - accuracy: 0.3274 - loss: 1.1051 - val_accuracy: 0.3910 - val_loss: 1.1088
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 10s 296ms/step - accuracy: 0.3058 - loss: 1.1176 - val_accuracy: 0.3383 - val_loss: 1.1010
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 10s 300ms/step - accuracy: 0.3492 - loss: 1.1018 - val_accuracy: 0.3308 - val_loss: 1.0979
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 12s 355ms/step - accuracy: 0.3324 - loss: 1.1005 - val_accuracy: 0.4135 - val_loss: 1.0921
Epoch 8/10
33/


Test Accuracy: 52.63%
Class indices: {'angular_leaf_spot': 0, 'bean_rust': 1, 'healthy': 2}


# Evaluation